## Final Assignment 

#### Part 1 and 2
1. The first part of AccessViz finds the from the data/TravelTimes folder the traveltimes-text files the user specifies with by YKR_ID in the YKR_ID_list in the start of the code.

2. After finding the specified traveltimes-txt files, this code reads the files and stores them into a folder data/shapefiles. The shapefiles are named the YKR_ID of the file. (eg. 5878087.shp)


In [1]:
import geopandas as gpd
import pandas as pd
import os

#read and take a look at the YKR_grid shp file 
grid_fp = 'data/MetropAccess_YKR_grid/MetropAccess_YKR_grid_EurefFIN.shp'

#read the file
grid = gpd.read_file(grid_fp)

# Change the name of YKR_ID to from_id, to enable the join
grid.rename(columns={'YKR_ID': 'from_id'}, inplace=True)

# See the column names and confirm that we now have a column called 'from_id'
grid.head()
grid.crs

{'proj': 'utm', 'zone': 35, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True}

1. This first part of the code finds all the textfiles in the data/TravelTime folder, and then store each of the traveltime modes into a shapefile named the YKR_ID number of the file. <br />

2. The user has to specify which YKR_IDs she/he wants to use in YKR_ID, where it says "specify YKR_IDs here!"<br />

3. The rest of the code reads in the .txt-files to pandad dataframe, converst it into a geopandas dataframe and stores each of the geodataframes into shapefiles called the YKR_ID number of the text file. 

4. I tried to write the code in a way that would say if the YKR_ID did not exist in the data/TravelTime-folder. My code would not work when I tried that, so i decided to leave it out, in order to make the code work. At the bottom of this markadown cell is the code I used for trying. I am pretty sure that the error is in how I coded the 'if'-statement, and I tried different ways, but non of the worked.

5. I also did not manage to write my code in a way that would inform the user about how the progression in the reading of files is going.

```
for YKR_ID in YKR_IDs: 
    #create path where the file is found
    path = ('data/TravelTimes/travel_times_to_')
    #create empty list where the filepath coresponding to YKR_ID-list can be stored.
    fp_list = []
    #create filepaths
    filepath = os.path.join(path + YKR_ID + '.txt')
    #append filepaths to the fp_list.
    fp_list.append(filepath)
    #print(fp_list)
    ```
```        
    #iterate over the filepaths stored in the fp_list
    for idx, fp in enumerate(fp_list):
        print(fp) 
    ```

```
'''if filepath does not exist in folder, print that the the filepath does not exist in the folder'''
        if fp is not in filepaths:
             print('This YKR_ID does not exist in folder')
             ```
```                
'''if the filepath exists in the filepaths list over files in the folder, create this into a dataframe'''
        else:
        data_ = pd.read_csv(fp, sep=';', usecols = ['from_id', 'pt_r_t'])
        print(data_.head())
```

In [2]:
#specify YKR_IDs here!
YKR_ID_list = ['5975373', '5878087']

In [3]:
import glob
import os
import geopandas as gpd
import pandas as pd
import numpy as np

#find all txt files in data/TravelTimes folder
all_filepaths = glob.glob('data/TravelTimes/*.txt')
#print(filepaths)

#list of YKR_IDs I want to import: 
YKR_IDs = YKR_ID_list

#create empty list where the filepath coresponding to YKR_ID-list can be stored.
fp_list = []

#Find filepaths for files specified in YKR_IDs 
for YKR_ID in YKR_IDs: 
    #create path where the file is found
    path = ('data/TravelTimes/travel_times_to_')
    #create filepaths
    filepath = os.path.join(path + YKR_ID + '.txt')

    #append filepaths to the fp_list.
    fp_list.append(filepath)
    #print(fp_list)
    #print(filepath)

In [ ]:
    #iterate over the filepaths stored in the fp_list
    for idx, fp in enumerate(fp_list):
        print(fp)
        
        #read the data to dataframe
        data_ = pd.read_csv(fp, sep=';')
        #print(data_.head())
        
        #change -1-values to NaN
        data_ = data_.replace(-1, np.nan)
        
        #getting rid of Nan-values
        #data_=data_.dropna()
        #print(data_.tail())
        
        #merge data_ (pd) and grid(gpd) together on 'from_id'  
        data = grid.merge(data_, on='from_id')
        #data.head()
   
        #set CRS
        CRS = grid.crs
        
        #make in into a geodataframe
        travel_times = gpd.GeoDataFrame(data, crs=CRS, geometry ='geometry')
        print(travel_times.head)
        
        #check the output
        #print(travel_times) 
        
        #save the geodataframes into shapefiles called the YKR_ID.shp in the folder shapefiles in data folder.
        for travel_time in travel_times.iterrows():
            #print(travel_time)
            # Create a output path for the data
            out_p = ('data/shapefiles/') 
            out_fp = os.path.join(out_p + YKR_ID + '.shp')
            travel_times.to_file(out_fp)

In [7]:
#print(travel_times.tail())

#### PART 3: 

This part of the code should visualise the shp-files stored in the previous part. It coould make both a static map and a interactive map based on a parameter the user passes in. This must be specified in the cell that had the header "specify travel mode and and type of map here". 0 for static map, 1 for interactive. 0 is set by default. 'pt_r_t' is set by default for travel mode. The user can change it to wished travelmode by following the link to the Helsinki Travel Times webside.

##### Issues with the code:
1. For some reason, which probably comes from storing of the code in the previous part, only the first part of the shapefiles is read into to the geodataframes. This results in that only half of the data is created on the maps, which makes  the maps rather uninformative. I have tried to fix this, but I did not manage to, and I therefor decided to leave it like this.

2. When it comes to the making the map it self, I found that my method is maybe a bit 'ineffective', especially when it comes to creating the interactive map. It would be a better solution to use 'mplleaflet.show()' after creating the static plt.plot()-map, but I did not find  this method before the very end, and it did not work the first time I tried it so decided to use how I solved it the first way for my code. 

In [4]:
import glob
import os
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
%matplotlib inline

shp_files = glob.glob('data/shapefiles/*.shp')
print(shp_files)


['data/shapefiles/5975373.shp', 'data/shapefiles/5878087.shp']


> #### SPECIFY TRAVELMODE AND TYPE OF MAP IN THIS CELL. 

Explanation for travelmode codes are found on this webpage: 
[https://blogs.helsinki.fi/accessibility/helsinki-region-travel-time-matrix-2018/]

In [12]:
#SPECIFY COLUMN YOU WOULD LIKE TO VISUALISE HERE. pt_r_t bu default.
column_for_visualising = "pt_r_t"

#Static or interactive. Replace 0 with 1 if interactive, static by default. 
map_type = 1

CRS = grid.crs
#print(CRS)

In [17]:
for idx, fp in enumerate(shp_files):
    #print(fp)
    shp_data = gpd.read_file(fp, crs=CRS )
    #print(shp_data.tail())
    #shp_data.crs
    
    if map_type > 0:
        #create interactive map
        # Re-project to WGS84
        shp_data_2 = shp_data.to_crs(epsg=4326)
        #shp_data.crs
        #print(shp_data.head())

        # Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
        shp_data_2['geoid'] = shp_data_2.index.astype(str)

        # Select only needed columns
        shp_data_2 = shp_data_2[['geoid', column_for_visualising, 'geometry']]
        
        # Create a Map instance
        int_map = folium.Map(location=[60.25, 24.8], tiles = 'cartodbpositron', zoom_start=10, control_scale=True)
        
        # Plot a choropleth map
        int_map.choropleth(
            geo_data=shp_data_2,
            name='Travel times Helsinki',
            data=shp_data_2,
            columns=['geoid', column_for_visualising, 'geometry'],
            key_on='feature.id',
            fill_color='YlOrRd',
            fill_opacity=0.7,
            line_opacity=0.2,
            line_color='white',
            line_weight=0,
            highlight=False,
            smooth_factor=1.0,
            #threshold_scale=[100, 250, 500, 1000, 2000],
            legend_name='Traveltimes to chosen YKR_ID in minutes')
        #checking the output
        #int_map
        
        #saving the map as html-file
        out_p_im = ('data/maps/')
        out_fp_im = os.path.join(out_p_im + YKR_ID + '.html')
        int_map.save(out_fp_im)


    else:
        
        shp_data_1 = shp_data.to_crs(epsg=3857)
        #create static map using matplotlib.plt
        #print(shp_data_1.tail())
        #print(shp_data_1)
        static_map = shp_data_1.plot(column=column_for_visualising, linewidth=0.03, figsize=(12, 8), cmap="RdYlBu", scheme="quantiles", k=7, alpha=0.6, legend=True)
    
        #get legend and place it at the map
        static_map.get_legend().set_title("Minutes with transport to YKR_ID by chosen transport")
        static_map.get_legend().set_bbox_to_anchor((1.0, 0.0))
    
        #set title on map
        static_map.set_title('Travel Times to specified YKR_ID')

        # Add basemap
        ctx.add_basemap(static_map, url = ctx.tile_providers.OSM_A)
        
        #remove empty white-space around the axes
        plt.tight_layout()
    
        # Save the figure as png file 
        out_p_sm = ('data/maps/')     
        out_fp_sm = os.path.join(out_p_sm + YKR_ID + '.png')
        plt.savefig(out_fp_sm)


 ##### Part 4
 compare traveltimes or distances between 2 different travel modes. 

##### Part 5 
Interactive map over the YKR grid values in Helsinki region. The purpose of the map is to help the user to choose the YKR-IDs that s/he is interested to visualize / analyze.

For this problem I guess we where supposed to make a map, looking like the Blokeh map on the Helsinki TravelTimes webpage. 

In [18]:
import geopandas as gpd
import pandas as pd
import os

#read and take a look at the YKR_grid shp file 
grid_fp = 'data/MetropAccess_YKR_grid/MetropAccess_YKR_grid_EurefFIN.shp'

#read the file
grid_1 = gpd.read_file(grid_fp)

# See the column names
grid_1.head()
#grid.crs

,x,y,YKR_ID,geometry
0,381875.0,6697880.0,5785640,"POLYGON ((382000.0001388059 6697750.000128186,..."
1,382125.0,6697880.0,5785641,"POLYGON ((382250.00013875 6697750.000128181, 3..."
2,382375.0,6697880.0,5785642,"POLYGON ((382500.0001386951 6697750.000128172,..."
3,382625.0,6697880.0,5785643,"POLYGON ((382750.0001386406 6697750.000128165,..."
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000138978 6697500.000128254, ..."


In [14]:
# Re-project to WGS84
grid_1 = grid_1.to_crs(epsg=4326)

# Check layer crs definition
print(grid_1.crs)

# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
grid_1['geoid'] = grid_1.index.astype(str)

# Select only needed columns
grid_1 = grid_1[['geoid', 'YKR_ID', 'geometry']]

#check data
#grid_1.tail()


{'init': 'epsg:4326', 'no_defs': True}


In [10]:
import folium
# Create a Map instance
YKRID_map = folium.Map(location=[60.25, 24.8], tiles = 'cartodbpositron', zoom_start=11, control_scale=True)

#YKRID_map

In [14]:
# Plot a choropleth map
YKRID_map.choropleth(
    geo_data=grid_1,
    name='Population in 2017',
    data=grid_1,
    columns=['geoid', 'YKR_ID'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white',
    line_weight=0,
    highlight=False,
    smooth_factor=1.0,
    threshold_scale=[5785640, 5800000, 5850000, 5900000, 5950000, 6018254], #[100, 250, 500, 1000, 2000],
    legend_name= 'YKR_ID cells, Helsinki')

#saving the map
out_YKRID_map_fp = "data/maps/YKR_ID_cells_Helsinki.html"
YKRID_map.save(out_YKRID_map_fp)

#Show map
#YKRID_map